In [1]:
import numpy as np
from gearbox_model import RxGearboxModel

## Tx Gearbox

In [2]:

header = [f' H{x}' for x in range(2)]
data = [f'D{x:02d}' for x in range(64)]
obuf = ['XXX' for _ in range(66)]


print('cnt / pause / head? / head_idx / data_idx / frame_word / obuf')

odata = []

for i in range(200):
    count = int(i % 33)
    pause = count == 32
    frame_word = int(count % 2 == 1)
    load_header = int(count % 2 == 0)
    header_idx = count if load_header else -1
    data_idx = count + 2 if load_header else count + 1


    
    # obuf[0:32] = obuf[32:64]
    # if not pause:
    #     if load_header:
    #         obuf[header_idx:header_idx+2] = header
    #     obuf[data_idx:data_idx+32] = data[frame_word*32 : 32 + frame_word*32]

    # odata.append(obuf[0:32])

    # bit select method
    for bit in range(len(obuf)):
        if bit < 32:
            obuf[bit] = obuf[bit+32]

        if not pause:
            if load_header:
                if bit >= header_idx and bit < header_idx + 2:
                    obuf[bit] = header[bit-header_idx]
                
            if bit >= data_idx and bit < data_idx + 32:
                if frame_word == 1:
                    obuf[bit] = data[(32 + bit) - data_idx]
                else:
                    obuf[bit] = data[bit - data_idx]


    odata.append(obuf[0:32])    

    print(f'{count:03d}\t{pause}\t{int(load_header)}\t{header_idx:02d}\t{data_idx:02d}\t{int(frame_word)}\t{obuf}')


cnt / pause / head? / head_idx / data_idx / frame_word / obuf
000	False	1	00	02	0	[' H0', ' H1', 'D00', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX']
001	False	0	-1	02	1	['D30', 'D31', 'D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39', 'D40', 'D41', 'D42', 'D43', 'D44', 'D45', 'D46', 'D47', 'D48', 'D49', 'D50', 'D51', 'D52', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D59', 'D60', 'D61', 'D62', 'D63', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 

## Rx Gearbox - no slip

In [3]:
rx_idata = odata.copy()

In [4]:
rxg = RxGearboxModel('str')

for id in rx_idata:
    rxg.next(id)
    print(rxg.get_state())


000	0	False	0	00	[' H0', ' H1']	['XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX']
001	0	True	1	32	[' H0', ' H1']	['D00', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31']
002	0	True	0	64	[' H0', ' H1']	['D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39', 'D40', 'D41', 'D42', 'D43', 'D44', 'D45', 'D46', 'D47', 'D48', 'D49', 'D50', 'D51', 'D52', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D59', 'D60', 'D61', 'D62', 'D63']
003	0	True	1	30	[' H0', ' H1']	['D00', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D2

## Rx Gearbox - half slip needed

In [7]:
# try with 1 slip out
rx_idata = odata.copy()
rx_idata = [item for sublist in rx_idata for item in sublist]
for _ in range(1): rx_idata.insert(0, 'XXX')

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

rx_idata = list(chunks(rx_idata, 32))[:-1]

In [8]:
rxg = RxGearboxModel('str')
slip_next = False

for id in rx_idata:
    ret = rxg.next(id, slip_next)
    print(rxg.get_state())
    slip_next = ret['header_valid'] and ret['header'] != [' H0', ' H1']

000	0	False	0	00	['XXX', ' H0']	['XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX']
001	0	True	1	32	['XXX', ' H0']	[' H1', 'D00', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30']
002	1	True	0	64	['D63', ' H0']	['D31', 'D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39', 'D40', 'D41', 'D42', 'D43', 'D44', 'D45', 'D46', 'D47', 'D48', 'D49', 'D50', 'D51', 'D52', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D59', 'D60', 'D61', 'D62']
002	0	True	0	64	['D30', 'D31']	['D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39', 'D40', 'D41', 'D42', 'D43', 'D44', 'D45', 'D46', 'D47', 'D48', 'D49', 'D50', 'D51', 'D52', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D59', 'D6